In [15]:
import pandas as pd
import re
import os,sys

# Load your DataFrame
df = pd.read_csv('../data/cleaned_csv.csv')

In [16]:
df.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,ምርጥ ዕቃ,@MerttEka,6828,️ የትኩስ ውሃ መያዢያ ከረጢት ️ 1.8 ሊትር ውሃ ይይዛል ️ ሙቀት ከሚ...,2024-10-01 10:39:48+00:00,photos/@MerttEka_6828.jpg
1,ምርጥ ዕቃ,@MerttEka,6825,️የፍሪጅ ማስቀመጫ ️ እስከ 200 ኪሎ ድረስ ዕቃ የሚሸከም ️ ወፍራም ብ...,2024-09-30 13:19:34+00:00,photos/@MerttEka_6825.jpg
2,ምርጥ ዕቃ,@MerttEka,6822,️ የበር ስር ጫማ ማስቀመጫ ️ ወፍራም የብረት ቱቦ ️ጠንካራ ረጅም ጊዜ ...,2024-09-30 11:11:45+00:00,photos/@MerttEka_6822.jpg
3,ምርጥ ዕቃ,@MerttEka,6819,️ ዘመናዊ በኤሌክትሮኒክስ የሚሰራ ሚዛን ️እስከ 50 ኪሎ ይመዝናል ️ በ...,2024-09-29 15:35:03+00:00,photos/@MerttEka_6819.jpg
4,ምርጥ ዕቃ,@MerttEka,6818,️ ቦርጭ የሚሰበስብ ️የቦርጭ ስብ የሚያቃጥል ️ ማራኪ የሰውነት ቅርፅ ይ...,2024-09-29 15:13:37+00:00,photos/@MerttEka_6818.jpg


In [ ]:
# Define a function to label messages with entity types
def label_message_utf8_with_birr(message):
    if not isinstance(message, str):  # Check if the message is a string
        return "O"  # Return a default label for non-string entries

    labeled_tokens = []
    
    # Tokenize the message
    tokens = re.findall(r'\S+', message)
    
    for token in tokens:
        # Check if token is a price (e.g., 500 ETB, $100, or ብር)
        if re.match(r'^\d+(\.\d{1,2})?$', token) or 'ETB' in token or 'ዋጋ' in token or '$' in token or 'ብር' in token:
            labeled_tokens.append(f"{token} B-PRICE")  # Label as B-PRICE
        # Check if token could be a location (e.g., cities or general location names)
        elif any(loc in token for loc in ['Addis Ababa', 'ለቡ', 'ለቡ መዳህኒዓለም', 'መገናኛ', 'ቦሌ', 'ሜክሲኮ']):
            labeled_tokens.append(f"{token} B-LOC")  # Label as B-LOC
        # Assume other tokens are part of a product name or general text
        else:
            labeled_tokens.append(f"{token} O")  # Outside any entity
            
    return "\n".join(labeled_tokens)

# Apply the labeling function to the 'Message' column
df['Labeled_Message'] = df['Message'].apply(label_message_utf8_with_birr)


In [20]:
# Display the labeled messages
print("Sampled Labeled Messages:")
print(df[['Message', 'Labeled_Message']])

# Save the labeled dataset to a file in CoNLL format
labeled_data_birr_path = 'labeled_telegram_product_price_location.conll'
with open(labeled_data_birr_path, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        f.write(f"{row['Labeled_Message']}\n\n")

print(f"Labeled data saved to {labeled_data_birr_path}")

Sampled Labeled Messages:
                                                Message  \
0     ️ የትኩስ ውሃ መያዢያ ከረጢት ️ 1.8 ሊትር ውሃ ይይዛል ️ ሙቀት ከሚ...   
1     ️የፍሪጅ ማስቀመጫ ️ እስከ 200 ኪሎ ድረስ ዕቃ የሚሸከም ️ ወፍራም ብ...   
2     ️ የበር ስር ጫማ ማስቀመጫ ️ ወፍራም የብረት ቱቦ ️ጠንካራ ረጅም ጊዜ ...   
3     ️ ዘመናዊ በኤሌክትሮኒክስ የሚሰራ ሚዛን ️እስከ 50 ኪሎ ይመዝናል ️ በ...   
4     ️ ቦርጭ የሚሰበስብ ️የቦርጭ ስብ የሚያቃጥል ️ ማራኪ የሰውነት ቅርፅ ይ...   
...                                                 ...   
4163                                               2500   
4164                                 ዋጋ 2500 0983063957   
4165  የሙያ ባለቤት መሆን መሠልጠን ነው። ቀለም ቀቢ ሳያስፈልግዎ ቤትዎን ወይን...   
4166  ቤትና ግቢዎን እንዲሁም የብረት እና የእንጨት ቁሳቁስዎን ቀለም ቀቢ ሳያስ...   
4167                                          ለማዘዝ ://.   

                                        Labeled_Message  
0     ️ O\nየትኩስ O\nውሃ O\nመያዢያ O\nከረጢት O\n️ O\n1.8 B-...  
1     ️የፍሪጅ O\nማስቀመጫ O\n️ O\nእስከ O\n200 B-PRICE\nኪሎ ...  
2     ️ O\nየበር O\nስር O\nጫማ O\nማስቀመጫ O\n️ O\nወፍራም O\n...  
3     ️ O\nዘመናዊ O\nበኤሌክትሮኒክስ O\nየ